This script sends several identical responces to the Claude server in the loop with asking to return Python dictionary. Information from dictionaries is collected in dataFrame and is saved to disk.

In [1]:
import anthropic
import base64
import json
import pandas as pd

In [2]:
# pip install anthropic

In [3]:
# get api_key from file (it is saved in format "api_key: <your_api_key> #comment")
with open('api_key.txt') as fh:
    api_key = fh.readline() \
                     .split(':')[1].split()[0]


client = anthropic.Anthropic(api_key = api_key)

# api_key

In [4]:
# read the image file from disk and encode it to base64 string
def upload_image_from_disk(path_to_image):
    
    with open(path_to_image, "rb") as image_file:
        encoded_image = base64.b64encode(image_file.read()).decode('utf-8')

    return encoded_image


path_to_image = 'input\\Life expectancy in mice.png'
encoded_image = upload_image_from_disk(path_to_image)
# encoded_image[-100:]

<br>
<br>

In [17]:
# The function sends a request to the Claude server and get a response
def prompt_Claude(client,
                  system: str,
                  prompt: str,
                  encoded_image: "base64 string"):

    response = client.messages.create(
        model="claude-3-5-sonnet-20240620",
        max_tokens=1000,
        temperature=0,
        system=system,
        messages=[
            {
                "role": "user",
                "content": [
                    {
                        "type": "text",
                        "text": prompt
                    },
                    {
                        "type": "image",
                        "source": {
                            "type": "base64",
                            "media_type": "image/png",
                            "data": encoded_image
                        }
                    }
                ]
            }
        ]
    )

    return response


# system text and prompt text to send
system = "You are an expert in Kaplan-Meier curves."
prompt = "That is a chart of life expectancy for 3 groups of mice. For each group, what is the age at which survival reaches 15%? " + \
          "Your answer must be only unrounded values in the form of a Python dictionary."


# send request to Claude server several times and collect results in datFrame
df = pd.DataFrame()

number_tests = 4
for count in range(1, number_tests+1):
    print(f"{count}. ", end='')  # debugging: be sure that the iteration began
    
    response = prompt_Claude(client=client, system=system, prompt=prompt, encoded_image=encoded_image)
    response_text = response.content[0].text
    print(response_text)  # debugging: look at response

    # convert text-response to dictionary object
    dd = json.loads(response_text.replace('\'', '"'))

    # convert dictionary to dataFrame and combine with earlier responses
    df_current = pd.DataFrame(data=dd, index=[count])
    df = pd.concat([df, df_current], axis='index')


# save result to dist in the form of CSV-spreadsheet
df.to_csv('output\\result of experiment.csv')

df

1. {"TRE-OSK": 155.7142857142857, "Control-Dox": 140.7142857142857, "Jax historical": 145.7142857142857}
2. {"TRE-OSK": 155.7142857142857, "Control-Dox": 140.7142857142857, "Jax historical": 145.7142857142857}
3. {"TRE-OSK": 155.7142857142857, "Control-Dox": 140.7142857142857, "Jax historical": 145.7142857142857}
4. {"TRE-OSK": 155.7142857142857, "Control-Dox": 140.7142857142857, "Jax historical": 145.7142857142857}


,TRE-OSK,Control-Dox,Jax historical
1,155.714286,140.714286,145.714286
2,155.714286,140.714286,145.714286
3,155.714286,140.714286,145.714286
4,155.714286,140.714286,145.714286
